# DBM Mask R-CNN Code

*Upload the unzipped version of the DBM Mask RCNN folder into your Google Drive's "My Drive" before running the code for the first time, or if any of the files inside the folder has been updated*

In [ ]:
!pip uninstall keras-nightly
!pip install h5py==2.10.0

Uninstalling keras-nightly-2.5.0.dev2021032900:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/keras/*
    /usr/local/lib/python3.7/dist-packages/keras_nightly-2.5.0.dev2021032900.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/keras/applications/resnet50.py
    /usr/local/lib/python3.7/dist-packages/keras/engine/network.py
    /usr/local/lib/python3.7/dist-packages/keras/engine/topology.py
    /usr/local/lib/python3.7/dist-packages/keras/initializers.py
    /usr/local/lib/python3.7/dist-packages/keras/layers/experimental/__init__.py
    /usr/local/lib/python3.7/dist-packages/keras/layers/experimental/preprocessing/__init__.py
    /usr/local/lib/python3.7/dist-packages/keras/objectives.py
    /usr/local/lib/python3.7/dist-packages/keras/optimizers/__init__.py
    /usr/local/lib/python3.7/dist-packages/keras/optimizers/schedules/__init__.py
    /usr/local/lib/python3.7/dist-packages/keras/utils/test_utils.py
Proceed (y/n)?

### Importing the required libraries for Weights & Biases integration and for Mask R-CNN code

In [ ]:
import os
import numpy as np
!pip install tensorflow==1.15.0rc2
import tensorflow as tf
!pip install keras==2.2.5

     |████████████████████████████████| 412.3MB 43kB/s 
     |████████████████████████████████| 512kB 45.6MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 3.8MB 45.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=3be0d299717eff0047b38e8abcb35662491541d1f0f96fbf0d4e27c2b5b4671a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.13.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0rc2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninst

### Installing and logging into Weights & Biases account

In [ ]:
%%capture
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

!wandb login

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Mounting drive to upload files from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


### Uploading DBM dataset file (with images and XML files) and visualize, utils & model Python files

In [ ]:
!unzip "//gdrive//My Drive//DBM Mask RCNN//Updated New Data resized images.zip"
!cp "/gdrive/My Drive/DBM Mask RCNN/visualize_saveimage_orderedfilename.py" "visualize.py"
!cp "/gdrive/My Drive/DBM Mask RCNN/utils.py" "utils.py"
!cp "/gdrive/My Drive/DBM Mask RCNN/model_savetodrive.py" "model.py"

Archive:  //gdrive//My Drive//DBM Mask RCNN//Updated New Data resized images.zip
   creating: Updated New Data resized images/annots/
  inflating: Updated New Data resized images/annots/1.xml  
  inflating: Updated New Data resized images/annots/10.xml  
  inflating: Updated New Data resized images/annots/100.xml  
  inflating: Updated New Data resized images/annots/101.xml  
  inflating: Updated New Data resized images/annots/102.xml  
  inflating: Updated New Data resized images/annots/103.xml  
  inflating: Updated New Data resized images/annots/104.xml  
  inflating: Updated New Data resized images/annots/105.xml  
  inflating: Updated New Data resized images/annots/106.xml  
  inflating: Updated New Data resized images/annots/107.xml  
  inflating: Updated New Data resized images/annots/108.xml  
  inflating: Updated New Data resized images/annots/109.xml  
  inflating: Updated New Data resized images/annots/11.xml  
  inflating: Updated New Data resized images/annots/110.xml  
  

### Importing the required libraries for Mask R-CNN code

In [ ]:
import xml.etree
from numpy import zeros, asarray
from random import sample

!pip install mrcnn
import mrcnn
import mrcnn.config
import visualize
import utils
import model

     |████████████████████████████████| 61kB 5.6MB/s 
  Created wheel for mrcnn: filename=mrcnn-0.2-cp37-none-any.whl size=54930 sha256=f2dec17149d9f0c4e4f456f05578a1e5b8480924234271244eb72f7709c0ae23
  Stored in directory: /root/.cache/pip/wheels/11/ed/28/e550ddc897c04c336b923eae4eb35c9aae993d20ce39d9cc40
Successfully built mrcnn


Using TensorFlow backend.
wandb: WARNING Keras version 2.2.5 is not fully supported. Required keras >= 2.4.0


### DBM Configuration

In [ ]:
class DBMConfig(mrcnn.config.Config): 
    NAME = "DBM_cfg"

    BACKBONE = "resnet50"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    NUM_CLASSES = 2 #(background/non-DBM + DBM)

    STEPS_PER_EPOCH = 517

    PRE_NMS_LIMIT = 6000

    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)
    BACKBONE_STRIDES = [4, 8, 16, 32, 64]
    #IMAGE_MAX_DIM = 1024
    #IMAGE_MIN_DIM = 1024

    DETECTION_MIN_CONFIDENCE = 0.5
  
DBM_config = DBMConfig()
DBM_config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTAN

### DBM Dataset

In [ ]:
randimage_id = []
file = open("/gdrive/My Drive/DBM Mask RCNN/test_set_images_e2098524-1c3c-4198-8328-08700497ff67.txt", "r")
file_lines = file.read()
file_lines = file_lines.rstrip("\n")
array = file_lines.split("\n")
for num in array:
  randimage_id.append(int(num))
print(randimage_id)

[682, 314, 446, 133, 696, 99, 328, 312, 70, 103, 612, 624, 731, 540, 33, 98, 241, 398, 487, 482, 494, 327, 615, 610, 536, 267, 590, 187, 120, 30, 577, 380, 608, 84, 457, 526, 708, 674, 362, 462, 41, 168, 701, 90, 262, 87, 435, 330, 80, 401, 255, 47, 519, 459, 722, 458, 335, 167, 93, 406, 499, 296, 374, 46, 293, 304, 269, 588, 719, 7, 403, 119, 210, 174, 259, 395, 632, 240, 424, 433, 206, 354, 217, 211, 359, 370, 189, 37, 323, 725, 391, 115, 476, 32, 460, 127, 505, 264, 205, 266, 135, 730, 539, 200, 214, 36, 324, 62, 668, 156, 626, 651, 146, 221, 106, 473, 45, 537, 454, 305, 216, 238, 474, 104, 514, 61, 298, 609, 636, 178, 611, 521, 507, 467, 176, 727, 400, 203, 376, 201, 180, 554, 392, 581, 165, 161, 385, 515, 310, 295, 704, 383, 652, 96, 321, 633, 152, 43, 732, 145, 155, 199, 402, 710, 182, 130, 441, 162, 273, 671, 74, 194, 281, 661, 530, 416, 520, 683, 197, 307, 22, 153, 690, 315, 509, 129, 644, 60, 2, 502, 228, 522, 557, 713, 83, 226, 503, 437, 69, 235, 343, 718, 63, 478, 114, 163, 

In [ ]:
class DBMDataset(utils.Dataset):
  
  def load_dataset(self, dataset_dir, is_train = True):
    self.add_class("dataset", 1, "DBM")
    
    images_dir = dataset_dir + '/images/'
    annotations_dir = dataset_dir + '/annots/'
    
    for filename in os.listdir(images_dir):
      image_id = filename.split('.')[0] # removes .jpg from the name of the file

      # skip all 80 randomly choosen images if we are building the train set
      # allow all the other images to be in the train set
      if is_train and int(image_id) in randimage_id:
        continue

      # allow all 80 randomly choosen images if we are building the train set,
      # skip all the other images to be in the training set
      if not is_train and int(image_id) not in randimage_id:
        continue
      
      img_path = images_dir + filename
      ann_path = annotations_dir + image_id + '.xml'
      
      self.add_image('dataset', image_id = image_id, path = img_path, annotation = ann_path)

  def extract_boxes(self, filename):
    tree = xml.etree.ElementTree.parse(filename)
    root = tree.getroot()
    
    boxes = list()
    for box in root.findall('.//bndbox'):
      xmin = int(box.find('xmin').text)
      ymin = int(box.find('ymin').text)
      xmax = int(box.find('xmax').text)
      ymax = int(box.find('ymax').text)
      coors = [xmin, ymin, xmax, ymax]
      boxes.append(coors)
    
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    return boxes, width, height
    
  def load_mask(self, image_id):
    info = self.image_info[image_id]
    path = info['annotation']
    boxes, w, h = self.extract_boxes(path)
    masks = zeros([h, w, len(boxes)], dtype = 'uint8')

    class_ids = list()
    for i in range(len(boxes)):
      box = boxes[i]
      row_s, row_e = box[1], box[3]
      col_s, col_e = box[0], box[2]
      masks[row_s:row_e, col_s:col_e, i] = 1
      class_ids.append(self.class_names.index('DBM'))
    return masks, asarray(class_ids, dtype = 'int32')

### Prepare training and validation sets

In [ ]:
train_set = DBMDataset()
train_set.load_dataset(dataset_dir = 'Updated New Data resized images', is_train = True)
train_set.prepare()

valid_dataset = DBMDataset()
valid_dataset.load_dataset(dataset_dir = 'Updated New Data resized images', is_train = False)
valid_dataset.prepare()

### Initializing and configuring Weights & Biases with the training run

In [ ]:
run = wandb.init(project='mask-rcnn-testing')

wandb: Currently logged in as: ronithsaju (use `wandb login --relogin` to force relogin)


### Loading the model

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
DBM_model = model.MaskRCNN(mode = "inference", config = DBM_config, model_dir = './')
DBM_model.load_weights(filepath = '/gdrive/My Drive/DBM Mask RCNN/Config Files/dbm_cfg20210713T1652/mask_rcnn_dbm_cfg_0008.h5', by_name = True)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead

Instructions for updating:
Use `tf.cast` instead.
Re-starting from epoch 8


### Displaying and saving all the results into DBM Model Results folder (inside DBM Mask RCNN folder), and calculating mAP

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
image_id = 0
DBM_count = 0
nonDBM_count = 0

for image_id in valid_dataset.image_ids:
  image, image_meta, gt_class_id, gt_bbox, gt_mask = model.load_image_gt(valid_dataset, DBM_config, image_id, use_mini_mask=False)
  print("image_id: ", image_id)
  info = valid_dataset.image_info[image_id]
  print(str(info["id"]))
  results = DBM_model.detect([image], verbose=0)
  r = results[0]

  if len(gt_bbox) != 0:

    gt_total_num = len(gt_bbox)
    gt_num = 0
    while gt_num < gt_total_num:
      x,y,w,h = gt_bbox[gt_num]
      gt_num += 1
      GT = image[x:w, y:h]
      plt.imsave('/gdrive/My Drive/DBM Mask RCNN/DBM/%s-%s.png' % (str(info["id"]), str(gt_num)), GT)
      DBM_count += 1

    if len(r['rois']) != 0:
      roi_total_num = len(r['rois'])
      roi_num = 0
      overlaps = utils.compute_overlaps(gt_bbox, r['rois'])
      image_num = 0
      while roi_num < roi_total_num:
        if max(overlaps[:, roi_num]) < 0.1:
          x,y,w,h = r['rois'][roi_num]
          image_num += 1
          RoI = image[x:w, y:h]
          plt.imsave('/gdrive/My Drive/DBM Mask RCNN/nonDBM/%s-%s.png' % (str(info["id"]), str(image_num)), RoI)
          nonDBM_count += 1

        roi_num += 1

  else:
    if len(r['rois']) != 0:
      image_num = 0
      for roi in r['rois']:
        x,y,w,h = r['rois'][image_num]
        image_num += 1
        RoI = image[x:w, y:h]
        plt.imsave('/gdrive/My Drive/DBM Mask RCNN/nonDBM/%s-%s.png' % (str(info["id"]), str(image_num)), RoI)
        nonDBM_count += 1

image_id:  0
295
image_id:  1
692
image_id:  2
383
image_id:  3
550
image_id:  4
156
image_id:  5
43
image_id:  6
615
image_id:  7
522
image_id:  8
61
image_id:  9
362
image_id:  10
557
image_id:  11
203
image_id:  12
304
image_id:  13
651
image_id:  14
520
image_id:  15
610
image_id:  16
269
image_id:  17
119
image_id:  18
730
image_id:  19
7
image_id:  20
608
image_id:  21
612
image_id:  22
403
image_id:  23
503
image_id:  24
74
image_id:  25
174
image_id:  26
83
image_id:  27
127
image_id:  28
682
image_id:  29
264
image_id:  30
370
image_id:  31
176
image_id:  32
208
image_id:  33
210
image_id:  34
201
image_id:  35
462
image_id:  36
104
image_id:  37
406
image_id:  38
459
image_id:  39
235
image_id:  40
33
image_id:  41
30
image_id:  42
167
image_id:  43
240
image_id:  44
478
image_id:  45
129
image_id:  46
87
image_id:  47
402
image_id:  48
226
image_id:  49
668
image_id:  50
37
image_id:  51
238
image_id:  52
230
image_id:  53
165
image_id:  54
636
image_id:  55
194
image_id:  5

In [ ]:
image_id = 0

for image_id in train_set.image_ids:
  image, image_meta, gt_class_id, gt_bbox, gt_mask = model.load_image_gt(train_set, DBM_config, image_id, use_mini_mask=False)
  print("image_id: ", image_id)
  info = train_set.image_info[image_id]
  print(str(info["id"]))
  results = DBM_model.detect([image], verbose=0)
  r = results[0]

  if len(gt_bbox) != 0:

    gt_total_num = len(gt_bbox)
    gt_num = 0
    while gt_num < gt_total_num:
      x,y,w,h = gt_bbox[gt_num]
      gt_num += 1
      GT = image[x:w, y:h]
      plt.imsave('/gdrive/My Drive/DBM Mask RCNN/DBM/%s-%s.png' % (str(info["id"]), str(gt_num)), GT)
      DBM_count += 1

    if len(r['rois']) != 0:
      roi_total_num = len(r['rois'])
      roi_num = 0
      overlaps = utils.compute_overlaps(gt_bbox, r['rois'])
      image_num = 0
      while roi_num < roi_total_num:
        if max(overlaps[:, roi_num]) < 0.1:
          x,y,w,h = r['rois'][roi_num]
          image_num += 1
          RoI = image[x:w, y:h]
          plt.imsave('/gdrive/My Drive/DBM Mask RCNN/nonDBM/%s-%s.png' % (str(info["id"]), str(image_num)), RoI)
          nonDBM_count += 1

        roi_num += 1

  else:
    if len(r['rois']) != 0:
      image_num = 0
      for roi in r['rois']:
        x,y,w,h = r['rois'][image_num]
        image_num += 1
        RoI = image[x:w, y:h]
        plt.imsave('/gdrive/My Drive/DBM Mask RCNN/nonDBM/%s-%s.png' % (str(info["id"]), str(image_num)), RoI)
        nonDBM_count += 1

image_id:  0
538
image_id:  1
558
image_id:  2
170
image_id:  3
198
image_id:  4
188
image_id:  5
272
image_id:  6
567
image_id:  7
387
image_id:  8
631
image_id:  9
250
image_id:  10
419
image_id:  11
658
image_id:  12
202
image_id:  13
239
image_id:  14
26
image_id:  15
245
image_id:  16
192
image_id:  17
712
image_id:  18
596
image_id:  19
81
image_id:  20
11
image_id:  21
726
image_id:  22
302
image_id:  23
6
image_id:  24
583
image_id:  25
638
image_id:  26
280
image_id:  27
18
image_id:  28
448
image_id:  29
377
image_id:  30
663
image_id:  31
542
image_id:  32
316
image_id:  33
578
image_id:  34
575
image_id:  35
191
image_id:  36
54
image_id:  37
105
image_id:  38
91
image_id:  39
276
image_id:  40
158
image_id:  41
465
image_id:  42
288
image_id:  43
470
image_id:  44
283
image_id:  45
386
image_id:  46
251
image_id:  47
34
image_id:  48
551
image_id:  49
733
image_id:  50
55
image_id:  51
121
image_id:  52
102
image_id:  53
653
image_id:  54
275
image_id:  55
597
image_id:  5

In [ ]:
print(DBM_count)
print(nonDBM_count)

11154
19840


In [ ]:
print(len(os.listdir("/gdrive/My Drive/DBM Mask RCNN/DBM")))
print(len(os.listdir("/gdrive/My Drive/DBM Mask RCNN/nonDBM")))

11154
19840


In [ ]:
wandb.finish()